## Defining a Database in pure Python

Design choices:
    - a table is a pair (attributes, instance)
        - attributes is a list of strings
        - instance is a list of rows
            - a row is a dictionary mapping attribute name to values

In [17]:
import json

def save(db, filename):
    with open(filename, 'w') as f:
        json.dump(db, f, sort_keys = True, indent = 4, ensure_ascii = False)

def load(filename):
    with open(filename, 'r') as f:
        return json.load(f)

In [1]:
def create(attributes):
    """ create an empty table with given attributes """
    return (attributes.copy(), [])

In [2]:
def add(table, row):
    """ add a row to the table """
    (attributes, instance) = table
    instance.append(row)

In [3]:
def add_copy(table, row):
    """ add a row to the table """
    (attributes, instance) = table
    instance.append(row.copy())

In [4]:
def pretty_print(table):
    """ print a table """
    (attributes, instance) = table
    print(','.join(attributes))
    for row in instance:
        print(','.join([str(row[k]) for k in attributes]))

In [5]:
def project(table, atts):
    """ project a table over a list of attributes """
    (attributes, instance) = table
    result = create(atts)
    for row in instance:
        new_row = { att:row[att]  for att in atts}
        add(result, new_row)
    return result

In [6]:
def select(table, att, value):
    """ select rows for which att==value is true """
    (attributes, instance) = table
    result = create(attributes)
    for row in instance:
        if row[att] == value:
            add_copy(result, row )
    return result

In [7]:
def select_ext(table, cond):
    """ select rows for which a boolean condition is true """
    (attributes, instance) = table
    result = create(attributes)
    for row in instance:
        if cond(row):
            add_copy(result, row )
    return result

In [8]:
def sort(table, atts = None):
    """ sort rows according to lexicographic ordering of attributes in atts """
    (attributes, instance) = table
    """ sorted(x, k) returns a sorted copy of x, 
            where comparison is made on k(e) for e in x
        since comparison between dictionaries is not possibile,
            we compare the list of values of a dictionary
    """
    if atts == None:
        atts = attributes
    sorted_inst = sorted(instance, key = lambda row: [row[k] for k in atts])
    return (attributes.copy(), sorted_inst)

In [9]:
def distinct(table):
    """ select distinct rows """
    (attributes, sorted_inst) = sort(table)
    result = create(attributes)
    previous = None
    for row in sorted_inst:
        if row != previous:
            add_copy(result, row)
        previous = row
    return result

In [10]:
def rename(table, att_a, att_b):
    """ rename an attribute A to B """
    (attributes, instance) = table
    result = create(attributes)
    (atts, inst) = result
    pos = atts.index(att_a)
    atts[pos] = att_b
    for row in instance:
        row_copy = row.copy()
        row_copy[att_b] = row[att_a]
        row_copy.pop(att_a) # remove key att_a
        add(result, row_copy)
    return result

In [11]:
def merge_two_dicts(d1, d2):
    d = d1.copy()
    d.update(d2)
    return d

In [12]:
def join(table1, table2, atts1, atts2):
    """ nested loop join of two tables on join condition atts1 = atts2 """
    (attributes1, instance1) = table1
    (attributes2, instance2) = table2
    result = create(attributes1+attributes2)
    for row1 in instance1:
        row_p1 = [ row1[att] for att in atts1 ]
        for row2 in instance2:
            row_p2 = [ row2[att] for att in atts2 ]
            if row_p1 == row_p2:
                add(result, merge_two_dicts(row1, row2))
    return result

In [13]:
def group_by(table, atts, count_att):
    """ group by atts -- and aggregate by COUNT(*) """
    (attributes, sorted_inst) = sort(table, atts)
    result = create(atts+[count_att])
    previous = None
    count = 0
    for row in sorted_inst:
        row_p = { att:row[att] for att in atts }
        if row_p != previous and count > 0:
            add(result, merge_two_dicts(previous, { count_att:count }) )
            count = 0
        previous = row_p
        count += 1
    if count > 0:
        add(result, merge_two_dicts(previous, { count_att:count }) )
    return result

In [14]:
def union_all(table1, table2):
    """ union of two tables, whose schema are the same """
    (attributes1, instance1) = table1
    (attributes2, instance2) = table2
    return (attributes1.copy(), instance1 + instance2)

In [15]:
def intersect_all(table1, table2):
    """ intersection of two tables, whose schema are the same """
    (attributes1, sorted_inst1) = sort(table1)
    (attributes2, sorted_inst2) = sort(table2)
    result = create(attributes1)
    result = []
    n1 = len(sorted_inst1)
    pos1 = 0
    n2 = len(sorted_inst2)
    pos2 = 0
    while pos1 < n1 and pos2 < n2:
        row1 = sorted_inst1[pos1]
        row2 = sorted_inst2[pos2]
        row1_l = [ row1[k] for k in attributes1 ]
        row2_l = [ row2[k] for k in attributes2 ]
        if row1_l > row2_l:
            pos2 += 1
        elif row1_l < row2_l:
            pos1 += 1
        else:
            add(result, row1)
            pos1 += 1
            pos2 += 1
    return result

In [16]:
def difference_all(table1, table2):
    """ difference of two tables, whose schema are the same """
    (attributes1, sorted_inst1) = sort(table1)
    (attributes2, sorted_inst2) = sort(table2)
    result = create(attributes1)
    n1 = len(sorted_inst1)
    pos1 = 0
    n2 = len(sorted_inst2)
    pos2 = 0
    while pos1 < n1 and pos2 < n2:
        row1 = sorted_inst1[pos1]
        row2 = sorted_inst2[pos2]
        row1_l = [ row1[k] for k in attributes1 ]
        row2_l = [ row2[k] for k in attributes2 ]
        if row1_l > row2_l:
            pos2 += 1
        elif row1_l == row2_l:
            pos1 += 1
            pos2 += 1
        else:
            add(result, row1)
            pos1 += 1
    while pos1 < n1:
        row1 = sorted_inst1[pos1]
        add(result, row1)
        pos1 += 1
    return result

## Examples
### Table definition, select, project, distinct, group by, rename

In [21]:
students = create([ 'Name', 'StudCode', 'City', 'BirthYear' ])
add(students, { 'Name':'Isaia', 'StudCode':171523, 'City':'PI', 'BirthYear':1982 } )
add(students, { 'Name':'Isaia', 'StudCode':171523, 'City':'PI', 'BirthYear':1982 } )
add(students, { 'Name':'Rossi', 'StudCode':167459, 'City':'LU', 'BirthYear':1980 } )
add(students, { 'Name':'Bianchi', 'StudCode':179856, 'City':'LI', 'BirthYear':1981 } )
add(students, { 'Name':'Bonini', 'StudCode':175649, 'City':'PI', 'BirthYear':1982 } )
pretty_print(students)

Name,StudCode,City,BirthYear
Isaia,171523,PI,1982
Isaia,171523,PI,1982
Rossi,167459,LU,1980
Bianchi,179856,LI,1981
Bonini,175649,PI,1982


In [37]:
pretty_print(select(students, 'City', 'PI') )

Name,StudCode,City,BirthYear
Isaia,171523,PI,1982
Isaia,171523,PI,1982
Bonini,175649,PI,1982


In [38]:
pretty_print(project(select(students, 'City', 'PI'), ['Name', 'City'] ) )

Name,City
Isaia,PI
Isaia,PI
Bonini,PI


In [20]:
q1 = distinct( project( select(students, 'City', 'PI'), ['Name', 'City'] ) )
pretty_print(q1)

Name,City
Bonini,PI
Isaia,PI


In [22]:
q2 = distinct( project( 
    select_ext(students, lambda row: row['City']=='PI' or row['StudCode']%2 == 1), ['Name', 'City'] ) )
pretty_print(q2)

Name,City
Bonini,PI
Isaia,PI
Rossi,LU


In [23]:
q3 = group_by(students, ['BirthYear'], 'N')
pretty_print(q3)

BirthYear,N
1980,1
1981,1
1982,3


In [24]:
q4 = rename(q3, 'BirthYear', 'Year of Birth')
pretty_print(q4)

Year of Birth,N
1980,1
1981,1
1982,3


### Join, Union and difference of tables

In [25]:
exams = create([ 'Subject', 'Candidate', 'Grade', 'Date' ])
add(exams, { 'Subject':'BD', 'Candidate':171523, 'Grade':20, 'Date':'12/01/05' } )
add(exams, { 'Subject':'ALG', 'Candidate':167459, 'Grade':30, 'Date':'15/09/05' } )
add(exams, { 'Subject':'MP', 'Candidate':171523, 'Grade':30, 'Date':'25/10/05' } )
add(exams, { 'Subject':'IS', 'Candidate':167459, 'Grade':20, 'Date':'10/10/05' } )
pretty_print(exams)

Subject,Candidate,Grade,Date
BD,171523,20,12/01/05
ALG,167459,30,15/09/05
MP,171523,30,25/10/05
IS,167459,20,10/10/05


In [26]:
q5 = join(students, exams, ['StudCode'], ['Candidate'])
pretty_print(q5)

Name,StudCode,City,BirthYear,Subject,Candidate,Grade,Date
Isaia,171523,PI,1982,BD,171523,20,12/01/05
Isaia,171523,PI,1982,MP,171523,30,25/10/05
Isaia,171523,PI,1982,BD,171523,20,12/01/05
Isaia,171523,PI,1982,MP,171523,30,25/10/05
Rossi,167459,LU,1980,ALG,167459,30,15/09/05
Rossi,167459,LU,1980,IS,167459,20,10/10/05


In [27]:
q6 = union_all(students, students)
pretty_print(q6)

Name,StudCode,City,BirthYear
Isaia,171523,PI,1982
Isaia,171523,PI,1982
Rossi,167459,LU,1980
Bianchi,179856,LI,1981
Bonini,175649,PI,1982
Isaia,171523,PI,1982
Isaia,171523,PI,1982
Rossi,167459,LU,1980
Bianchi,179856,LI,1981
Bonini,175649,PI,1982


In [29]:
q7 = difference_all(q6, students)
pretty_print(q7)

Name,StudCode,City,BirthYear
Bianchi,179856,LI,1981
Bonini,175649,PI,1982
Isaia,171523,PI,1982
Isaia,171523,PI,1982
Rossi,167459,LU,1980


### Saving and Loading the DB in Json Format

In [31]:
database = { 'students':students, 'exams':exams}
pretty_print(database['exams'])

Subject,Candidate,Grade,Date
BD,171523,20,12/01/05
ALG,167459,30,15/09/05
MP,171523,30,25/10/05
IS,167459,20,10/10/05


In [34]:
save(database, 'data/db.json')

In [36]:
db2 = load('data/db.json')
pretty_print(db2['exams'])

Subject,Candidate,Grade,Date
BD,171523,20,12/01/05
ALG,167459,30,15/09/05
MP,171523,30,25/10/05
IS,167459,20,10/10/05
